In [1]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"

In [2]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [3]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [4]:
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()

In [5]:
llm = ChatOpenAI(temperature=0) # Also works well with Anthropic models
agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [6]:
response = await agent_chain.arun(input="Hi I'm Erica.")
print(response)



> Entering new  chain...


Hello Erica! How can I assist you today?

> Finished chain.
Hello Erica! How can I assist you today?


In [7]:
response = await agent_chain.arun(input="Don't need help really just chatting.")
print(response)



> Entering new  chain...


That's perfectly fine! I'm here to chat with you. How's your day going?

> Finished chain.
That's perfectly fine! I'm here to chat with you. How's your day going?


In [8]:
response = await agent_chain.arun(input="Browse to blog.langchain.dev and summarize the text, please.")
print(response)



> Entering new  chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Observation: Navigating to https://blog.langchain.dev returned status code 200
Thought:I have successfully navigated to the blog.langchain.dev website. Now I need to extract the text from the webpage to summarize it.
Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```
Observation: LangChain LangChain Home About GitHub Docs LangChain The official LangChain blog. LangChain 🤝 Streamlit The initial integration of Streamlit with LangChain and our future plans. Jul 11, 2023 2 min read 🎉 Prem Challenge🎉 We're excited to announce a challenge hosted by Prem in collaboration with LangChain.

At LangChain we try to make it easy as possible to experiment with as many different models as possible. That includes the incredible number of emerging open source models. We've made efforts to make our framework as Jun 26, 2023 6 min read LangChain 

I have successfully navigated to the blog.langchain.dev website. The text on the webpage includes various blog posts about LangChain. Here are some of the titles and summaries of the blog posts:

1. "LangChain 🤝 Streamlit": This blog post discusses the initial integration of Streamlit with LangChain and their future plans.

2. "LangChain <> MongoDB Atlas": This post announces LangChain's integration with MongoDB Atlas, adding support for one of the most popular developer data platforms.

3. "Data-Driven Characters": This blog post introduces a repository for creating, debugging, and interacting with chatbots conditioned on story corpora.

4. "LangChain + Vectara: better together": The post announces the integration of Vectara into LangChain, allowing for fetching relevant documents based on user queries.

5. "GPTeam: A multi-agent simulation": This blog post highlights the GPTeam project, which incorporates key ideas from the Generative Agents paper.

6. "Going Beyond Chatbots: How to 

In [9]:
response = await agent_chain.arun(input="What's the latest xkcd comic about?")
print(response)



> Entering new  chain...


I can navigate to the xkcd website and check the latest comic for you. Should I do that?

> Finished chain.
I can navigate to the xkcd website and check the latest comic for you. Should I do that?


In [10]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [11]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [12]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"]
    }
)

In [13]:
response = await agent_chain.arun(input="Hi I'm Erica.")
print(response)



> Entering new  chain...


Hello Erica! How can I assist you today?

> Finished chain.
Hello Erica! How can I assist you today?


In [14]:
response = await agent_chain.arun(input="whats my name?")
print(response)



> Entering new  chain...


Your name is Erica.

> Finished chain.
Your name is Erica.
